Author: Alvin Thai
Per Scholas - index.ipynb manages the ETL process for a loan application and credit card dataset that using Python, MariaDB, Apache Spark, and Python Visualization libraries. 
8/30/2022

#### Data Extraction and Transformation with Python and PySpark


In [ ]:
from datetime import datetime
from datetime import date
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,FloatType

spark = SparkSession.builder.appName('capstone').getOrCreate()
branches = spark.read.json("cdw_sapp_branch.json")
customers = spark.read.json("cdw_sapp_customer.json")
credits = spark.read.json("cdw_sapp_credit.json")

branchRDD = branches.rdd.map(lambda x: (x[1], x[2], x[5], x[0], x[4], x[6], '('+x[3][:3]+')'+x[3][3:6]+'-'+x[3][6:], datetime.fromisoformat(x[7])))
branches = spark.createDataFrame(data=branchRDD, schema=["BRANCH_CODE", "BRANCH_NAME", "BRANCH_STREET", "BRANCH_CITY", "BRANCH_STATE", "BRANCH_ZIP", "BRANCH_PHONE", "LAST_UPDATED"])
branches.printSchema()
customerRDD = customers.rdd.map(lambda x: (int(x[12]), x[8].title(), x[11].lower(), x[9].title(), x[1], x[13]+", "+x[0], x[2], x[6], x[3], int(x[7]), str(x[5])[:3]+"-"+str(x[5])[3:], x[4], datetime.fromisoformat(x[10])))
customers = spark.createDataFrame(data=customerRDD, schema=["SSN","FIRST_NAME","MIDDLE_NAME", "LAST_NAME", "Credit_card_no", "FULL_STREET_ADDRESS", "CUST_CITY", "CUST_STATE", "CUST_COUNTRY", "CUST_ZIP", "CUST_PHONE", "CUST_EMAIL", "LAST_UPDATED"])
customers.printSchema()
creditRDD = credits.rdd.map(lambda x: (x[1], date(x[8], x[4], x[3]).strftime('%Y%m%d'), x[2], x[0], x[6], x[7], x[5]))  
credits = spark.createDataFrame(data=creditRDD, schema=["CUST_CC_NO", "TIMEID", "CUST_SSN", "BRANCH_CODE", "TRANSACTION_TYPE", "TRANSACTION_VALUE", "TRANSACTION_ID"])
credits.printSchema()

#### Data loading into Database

In [ ]:
branches.write.format("jdbc") \
  .mode("overwrite") \
  .option("url", "jdbc:mysql://localhost:3306/credit_capstone") \
  .option("dbtable", "credit_capstone.cdw_sapp_branch") \
  .option("user", "root") \
  .option("password", "root") \
  .save()
customers.write.format("jdbc") \
  .mode("overwrite") \
  .option("url", "jdbc:mysql://localhost:3306/credit_capstone") \
  .option("dbtable", "credit_capstone.cdw_sapp_customer") \
  .option("user", "root") \
  .option("password", "root") \
  .save()
credits.write.format("jdbc") \
  .mode("overwrite") \
  .option("url", "jdbc:mysql://localhost:3306/credit_capstone") \
  .option("dbtable", "credit_capstone.cdw_sapp_credit_card") \
  .option("user", "root") \
  .option("password", "root") \
  .save()

In [ ]:
branches.createTempView("branches")
customers.createTempView("customers")
credits.createTempView("credit_card")

#### View transactions from customer in given year and month within a zipcode ordered by day descending

In [ ]:
# year_month is in YYYYMM format.  Change the variables to see data from a different, month, and zipcode.
year_month = '201809'
zipcode = 55044

query = "SELECT t.TIMEID, t.TRANSACTION_ID, t.TRANSACTION_TYPE, t.TRANSACTION_VALUE \
          FROM branches b, customers c, credit_card t \
          WHERE b.BRANCH_CODE = t.BRANCH_CODE and t.CUST_CC_NO = c.Credit_card_no and t.TIMEID LIKE '" + year_month + "%' and c.CUST_ZIP = " + str(zipcode) + " \
          ORDER BY t.TIMEID DESC"
spark.sql(query).show(5)

#### Display the number and total values of transactions for a given type

In [ ]:
query = "SELECT t.TRANSACTION_TYPE, count(*) AS Orders, round(sum(t.TRANSACTION_VALUE), 2) AS Total \
          FROM credit_card t \
          GROUP BY t.TRANSACTION_TYPE"
spark.sql(query).show()

#### Display the number and total values of transactions for branches in a given state

In [ ]:
state = "NY"
query = "SELECT t.BRANCH_CODE as Branch, count(*) AS Orders, round(sum(t.TRANSACTION_VALUE), 2) AS Total \
          FROM credit_card t JOIN branches b on t.BRANCH_CODE = b.BRANCH_CODE \
          WHERE b.BRANCH_STATE = '" + state + "' \
          GROUP BY t.BRANCH_CODE"
spark.sql(query).show()

#### Check the existing account details of a customer

In [ ]:
ssn = 123456100
query = "SELECT * \
          FROM customers c \
          WHERE c.SSN = " + str(ssn) + ""
spark.sql(query).show()

#### Modify the existing account details of a customer

In [ ]:
import mysql.connector as mariadb
from mysql.connector import Error
from dotenv import dotenv_values

config = dotenv_values('.env')
try:
    connection = mariadb.connect(host='localhost',
                                         database='credit_capstone',
                                         user=config['user'],
                                         password=config['password'])
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to SQL  version ", db_Info)
        cursor = connection.cursor()
        SQLQuery ="UPDATE cdw_sapp_customer SET Credit_card_no = 4210653310061068 WHERE Credit_card_no = 4210653310061055";
        cursor.execute(SQLQuery)
        ssn = 123456100
        SQLQuery = "SELECT * \
                  FROM cdw_sapp_customer c \
                  WHERE c.SSN = " + str(ssn) + ""
        cursor.execute(SQLQuery)
        records = cursor.fetchall()
        print("Total number of rows from result table: ", cursor.rowcount)    
        print("\nPrinting each row")
        for row in records:
            print("SSN = ", row[0],  )
            print("First name = ", row[1])
            print("Middle name  = ", row[2])
            print("Last name = ", row[3])
            print("Credit card no. = ", row[4])
            print("Full Street Address = ", row[5])
            print("City = ", row[6])
            print("State = ", row[7])
            print("Country = ", row[8])
            print("Zipcode = ", row[9])
            print("Phone = ", row[10])
            print("Email = ", row[11])
            print("Last updated = ", row[12], "\n")
except Error as e:
        print("Error while connecting to Database", e)
finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("Database connection is closed")


#### Generate a monthly bill for a credit card number for a given month and year

In [ ]:
# cc_no and year_month changes the output.  cc_no is the credit card number and year_month is in YYYYMM format.
cc_no = "4210653349028689"
year_month = "201809"
query = "SELECT round(sum(t.TRANSACTION_VALUE), 2) \
          FROM credit_card t \
          WHERE t.CUST_CC_NO = '" + cc_no + "' and startswith(t.TIMEID, '" + year_month + "')"
spark.sql(query).show()

#### Display the transactions made by a customer between two dates ordered by year, month, and day in descending order

In [ ]:
cc_no = "4210653349028689"
start = "20180501"
end = "20180701"
spark.sql("SELECT t.TIMEID, t.TRANSACTION_ID, t.TRANSACTION_TYPE, t.TRANSACTION_VALUE \
          FROM credit_card t \
          WHERE t.CUST_CC_NO = '" + cc_no + "' and t.TIMEID > '" + start + "' and t.TIMEID < '" + end + "' \
          ORDER BY EXTRACT(YEAR FROM t.TIMEID), EXTRACT(MONTH FROM t.TIMEID), EXTRACT(DAY FROM t.TIMEID) DESC").show()

### Data Analysis and Visualization

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
%matplotlib inline

#### Which transaction type occurs most often

In [ ]:
purchases = credits.toPandas()
transaction_types = sns.countplot(data=purchases, y='TRANSACTION_TYPE')

#### Which state has the highest number of customers

In [ ]:
customers = customers.toPandas()
# ax = sns.countplot(data=customers, y="CUST_STATE")
count_df = customers['CUST_STATE'].value_counts().to_frame().reset_index().rename(columns={'index':'State', 'CUST_STATE':'Count'})
transactions_by_state = px.choropleth(count_df, locations=count_df['State'], locationmode='USA-states', scope='usa', color='Count', color_continuous_scale='tealgrn', title='Transactions by State')
transactions_by_state.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
transactions_by_state.show()

#### Sum of all transactions for each customer, and which customer has the highest transaction amount

In [ ]:
transaction_sums = purchases[['CUST_SSN', 'TRANSACTION_VALUE']].groupby('CUST_SSN').sum().rename(columns={"TRANSACTION_VALUE": "Sum"}).reset_index()
transaction_sums['CUST_SSN'] = transaction_sums['CUST_SSN'].astype(str)
cust_totals = px.scatter(transaction_sums, x='Sum', y='CUST_SSN', title='Transaction Totals by Customer', color='Sum', color_continuous_scale='oryel')
cust_totals.add_annotation(
    xref="x domain",
    yref="y domain",
    x=0.94,
    y=0.09,
    text="Highest Purchase",
    arrowhead=2,
)
cust_totals.update_layout(
    autosize=False,
    width=1500,
    height=800,)
cust_totals.show()

In [ ]:
spark.sql("SELECT SUBSTRING(t.TIMEID, 5, 2), round(sum(t.TRANSACTION_VALUE), 2) \
          FROM credit_card t \
          GROUP BY SUBSTRING(t.TIMEID, 5, 2) \
          ORDER BY sum(t.TRANSACTION_VALUE) DESC").show()

#### Top three months with the largest transaction data

In [ ]:
month_purchases = purchases[['TIMEID', 'TRANSACTION_VALUE']]
months = {'01': 'January', '02':'February', '03':'March', '04':'April', '05':'May', '06':'June', '07':'July', '08':'August', '09':'September', '10':'October', '11':'November', '12':'December'}
month_purchases['MONTH'] = month_purchases['TIMEID'].apply(lambda e: months[e[4:6]])
month_sums = month_purchases.groupby('MONTH').sum().rename(columns={"TRANSACTION_VALUE": "Sum"}).reset_index()
top_three = px.bar(month_sums.nlargest(n=3, columns=['Sum']), x='Sum', y='MONTH', labels={'Sum':'Total Purchases (Dollars)', 'MONTH':'Month'}, title="Top Three Months", orientation='h')
# show from most to least
top_three.update_layout(yaxis={'categoryorder':'total ascending'})

#### Healthcare transactions

In [ ]:
branch_spendings = purchases[['BRANCH_CODE', 'TRANSACTION_TYPE', 'TRANSACTION_VALUE']]
branch_spendings = branch_spendings.groupby(['TRANSACTION_TYPE', 'BRANCH_CODE']).sum().unstack()
branch_spendings = branch_spendings.loc['Healthcare'].to_frame().reset_index().drop(['level_0'], axis=1).rename(columns={"Healthcare": "BRANCH_SUM"})
branch_spendings
health_spend = px.scatter(branch_spendings, x='BRANCH_CODE', y='BRANCH_SUM', size='BRANCH_SUM', title="Healthcare Spendings (Dollars) by Branch")
health_spend.show()

### Import from API

In [ ]:
import requests
import json
response = requests.get("https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json")
print(response)

In [ ]:
loans = response.json()
loans = spark.createDataFrame(data=loans)
loans.printSchema()

In [ ]:
loans.write.format("jdbc") \
  .mode("overwrite") \
  .option("url", "jdbc:mysql://localhost:3306/credit_capstone") \
  .option("dbtable", "credit_capstone.cdw_sapp_loan_application") \
  .option("user", "root") \
  .option("password", "root") \
  .save()

In [ ]:
df = loans.toPandas()

In [ ]:
df.head()

#### Difference in application approvals for Married Men vs Married Women based on income ranges

In [ ]:
married_only = df[df['Married'] =='Yes']
approvals_only = married_only[married_only['Application_Status']=='Y']

In [ ]:
married_income_app = approvals_only[['Gender', 'Income']].value_counts().to_frame().rename(columns={0:'Count'}) \
.plot(kind='barh', title='Income-based Approvals between Married Men and Married Women for Credit')

#### Difference in application approvals based on Property Area

In [ ]:
prop_approval = df[df['Application_Status']=='Y']
prop_app = prop_approval[['Property_Area', 'Application_Status']].value_counts().to_frame().rename(columns={0:'Count'}).plot(kind='barh', title='Property Area based Approval for Credit')

In [ ]:
approved = df[df['Application_Status'] =='Y']

# fig, ax = plt.subplots()
# ax.plot(approved.groupby(['Credit_History']).count().reset_index()['Application_ID'].to_frame(), label='Credit_History')
# ax.plot(approved.groupby(['Dependents']).count().reset_index()['Application_ID'].to_frame(), label='Dependents')
# ax.plot(approved.groupby(['Education']).count().reset_index()['Application_ID'].to_frame(), label='Education')
# ax.plot(approved.groupby(['Income']).count().reset_index()['Application_ID'].to_frame(), label='Income')
# ax.plot(approved.groupby(['Married']).count().reset_index()['Application_ID'].to_frame(), label='Married')
# ax.plot(approved.groupby(['Property_Area']).count().reset_index()['Application_ID'].to_frame(), label='Property_Area')
# ax.plot(approved.groupby(['Self_Employed']).count().reset_index()['Application_ID'].to_frame(), label='Self_Employed')
# ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
# plt.show()

approved['Dependents'].value_counts().to_frame().plot(kind='bar')


In [ ]:
cred_app = sns.countplot(x='Application_Status', hue='Credit_History', data=df, palette='rainbow')

In [ ]:
dep_app = sns.countplot(x='Application_Status', hue='Dependents', data=df, palette='rainbow')

In [ ]:
edu_app = sns.countplot(x='Application_Status', hue='Education', data=df, palette='rainbow')

In [ ]:
income_app = sns.countplot(x='Application_Status', hue='Income', data=df, palette='rainbow')

In [ ]:
married_app = sns.countplot(x='Application_Status', hue='Married', data=df, palette='rainbow')

In [ ]:
sns.countplot(x='Application_Status', hue='Property_Area', data=df, palette='rainbow')

In [ ]:
emp_app = sns.countplot(x='Application_Status', hue='Self_Employed', data=df, palette='rainbow')

In [ ]:
self_employed = pd.get_dummies(df['Self_Employed'], drop_first=True).rename(columns={'Yes': 'self_employed'})
property_area = pd.get_dummies(df['Property_Area'])
application_status = pd.get_dummies(df['Application_Status'], drop_first=True)
married = pd.get_dummies(df['Married'], drop_first=True).rename(columns={'Yes': 'married'})
income = pd.get_dummies(df['Income'], drop_first=True)
education = pd.get_dummies(df['Education'], drop_first=True)
dependents = pd.get_dummies(df['Dependents'])
credit_history = pd.get_dummies(df['Credit_History'], drop_first=True)
gender = pd.get_dummies(df['Gender'], drop_first=True)

In [ ]:
X = pd.concat([self_employed, married, education, credit_history, gender], axis=1).rename(columns={'Yes': 'self_employed_yes', 'Yes':'married_yes', 1:'credit_history_1'})
y = application_status

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train.values.ravel())

In [ ]:
predictions = logmodel.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

In [ ]:
import dash
from dash import html
from dash import dcc

In [ ]:
app = dash.Dash(__name__)
app.layout = html.Div(children=[html.H1('Credit Visualization',
                                    style={'textAlign': 'center',
                                        'color': '#503D36',
                                        'font-size': 40}),
                                html.P('How do demographics affect credit acceptance?',
                                    style={'textAlign':'center', 'color': '#F57241'}),
                                dcc.Graph(figure=transactions_by_state),
                                dcc.Graph(figure=cust_totals),
                                dcc.Graph(figure=top_three),
                                dcc.Graph(figure=health_spend),
                    ])
app.run_server()

In [ ]:
spark.stop()